In [1]:
from pre_processing import *
from syllable_dictionary import *
from split_poem import *
import os
import numpy as np
from IPython.display import HTML
import heapq

from HMM import unsupervised_HMM
from HMM_helper import (
    text_to_wordcloud,
    states_to_wordclouds,
    parse_observations,
    sample_sentence,
    visualize_sparsities,
    animate_emission
)
import random
from build_rhyme_dict import *
from syllable_dictionary import *

[nltk_data] Downloading package punkt to /Users/ywwu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# get syllable dictionary
syllable_filename = "./project3/data/Syllable_dictionary.txt"
syllable_dict, syllable_end_dict = get_syllable_dict(syllable_filename)

In [3]:
# get rhyme dictionary
rhyme_filename = "./project3/data/shakespeare.txt"
rhyme_dict = get_rhyme_dict(rhyme_filename)

In [4]:
# run HMM
filename = "./project3/data/shakespeare.txt"
obs_s, obs_map_s = pre_processing_sentences_reverse(filename)
# get reversed obs_map_s
obs_map_s_rev = {}
for key, value in obs_map_s.items():
    obs_map_s_rev[value] = key

count of puncs =  0


In [ ]:
num_hidden_states = [5, 8, 10, 15, 20]
iterations = [50, 100, 150]
hmms = []
num_obs_displayed = 10
for num_hs in num_hidden_states:
    print("***************************** Hidden States Start! *****************************")
    print("Current number of hidden states = ", num_hs)
    
    for iteration in iterations:
        print("----------------------- Iteration Start! -----------------------")
        print("Current iteration times = ", iteration)
        hmm8_s = unsupervised_HMM(obs_s, num_hs, iteration)
        hmms.append(hmm8_s)
        
        curr_poem_output = generate_rhyme(hmm8_s, rhyme_dict, syllable_dict, syllable_end_dict, obs_map_s)
        print("+++++++++ Current Poem Start! ++++++++")
        print(curr_poem_output)
        print("+++++++++ Current Poem End! ++++++++++")
        
        state_obs_topN = get_high_prob_obs(hmm8_s, num_obs_displayed)
        for key, value in state_obs_topN.items():
            print("///State = ", key, ", top N prob obs: ", value, "///")
            
        print("----------------------- Iteration End! ------------------------")
        
    print("***************************** Hidden States End! *****************************")
        

# generated_sentence = sample_sentence(hmm8_s, obs_map_s, n_words=300)

***************************** Hidden States Start! *****************************
Current number of hidden states =  5
----------------------- Iteration Start! -----------------------
Current iteration times =  50
Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50


In [8]:
# generate final poem
#poem_output = generate_rhyme(hmm, rhyme_dict, syllable_dict, syllable_end_dict, obs_map_s)

KeyboardInterrupt: 

In [9]:
# words = ["about","about","a-doting","about","a","about"]
# valid, valid_sentence = exact_ten_syllables(words, syllable_dict, syllable_end_dict)
# print(valid)
# print(valid_sentence)

In [10]:
# visualize

In [23]:
def get_high_prob_obs(hmm, num_obs_displayed):
    A_mat = hmm.A
    O_mat = hmm.O
    state_obs_topN = collections.defaultdict(list)
    for i in range(len(O_mat)): # states, O_mat[i] are the probabilities of each obs under this state
        obs_prob = []
        for j in range(len(O_mat[i])): # observations
            heapq.heappush(obs_prob, (O_mat[i][j], obs_map_s_rev[j])) # (prob, obs-word)
        top_n = heapq.nlargest(num_obs_displayed, obs_prob)
        for prob, obs in top_n:
            state_obs_topN[i].append(obs)
#             print("************************** Start *******************************")
#             print("Current state = ", i)
#             print("Max_State_Prob_Trans: observation: ", obs, ", probability = ", prob)
#             print("************************** End *******************************")
    return state_obs_topN
  

In [24]:
# states - top N prob obs
num_obs_displayed = 5
state_obs_topN = get_high_prob_obs(hmm8_s, num_obs_displayed)
for key, value in state_obs_topN.items():
    print("State = ", key, ", top N prob obs: ", value)

State =  0 , top N prob obs:  ['my', 'i', 'the', 'and', 'of']
State =  1 , top N prob obs:  ['of', 'thou', 'with', 'my', 'in']
State =  2 , top N prob obs:  ['thy', 'that', 'i', 'of', 'and']
State =  3 , top N prob obs:  ['the', 'i', 'and', 'of', 'not']
State =  4 , top N prob obs:  ['to', 'the', 'my', 'not', 'and']
State =  5 , top N prob obs:  ['and', 'of', 'my', 'that', 'the']
State =  6 , top N prob obs:  ['my', 'the', 'to', 'and', 'thou']
State =  7 , top N prob obs:  ['of', 'to', 'love', 'and', 'i']
State =  8 , top N prob obs:  ['and', 'to', 'the', 'thy', 'that']
State =  9 , top N prob obs:  ['in', 'thy', 'and', 'to', 'that']


In [ ]:
# visualize_sparsities
visualize_sparsities(hmm8_s, O_max_cols=50)